In [0]:
!pip install pulp            # PuLP
!apt-get install glpk-utils  # GLPK
!apt-get install coinor-cbc  # CoinOR

from pulp import *
import pandas as pd
import numpy as np
import os

In [0]:
!git clone https://github.com/vaastav/Fantasy-Premier-League

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


## Constructing a data set to test on

In [0]:
ROOT_PATH = '/content/drive/My Drive/KTH   Studier/Åk 5/Scalable Machine Learning'
ROOT_PATH = '/content/drive/My Drive/skola/Scalable Machine Learning'
player_weeks = pd.read_csv(os.path.join(ROOT_PATH, 'final_results.csv'))
player_weeks = player_weeks.loc[:, ~player_weeks.columns.str.contains('^Unnamed')]
player_weeks = player_weeks[player_weeks['season'] == '2018-19']
player_weeks

,player_id,week,total_points,season,cost_pred,cost_real,long_pred,long_real,short_pred,short_real
1,1,1,0,2018-19,-0.012661,-4.666667,1.544490,0.894737,2.242678,1.666667
2,1,25,2,2018-19,-0.005661,1.333333,1.853603,0.918919,3.614124,3.000000
3,1,19,2,2018-19,0.003674,0.000000,1.148359,0.888889,2.846313,2.250000
4,1,37,0,2018-19,-0.006991,1.333333,0.500508,0.857143,1.528017,0.583333
5,1,6,0,2018-19,-0.008576,-2.666667,0.564175,0.882353,1.399779,0.833333
...,...,...,...,...,...,...,...,...,...,...
15374,865,25,2,2018-19,-0.009120,0.000000,0.853458,2.000000,1.801127,2.000000
15375,866,25,0,2018-19,0.001557,0.000000,0.632868,0.000000,1.119330,0.000000
15376,867,37,2,2018-19,0.012554,0.000000,2.531351,2.000000,4.955649,2.000000
15377,868,37,2,2018-19,0.004106,0.000000,1.003423,2.000000,4.630850,2.000000


In [0]:
all_gws = pd.read_csv(os.path.join(ROOT_PATH, 'processed_dataset.csv'))
all_gws = all_gws.loc[:, ~all_gws.columns.str.contains('^Unnamed')]
all_gws = all_gws[all_gws['season'] == '2018-19']
all_gws = all_gws[['player_index', 'week_raw', 'cost_raw', 'team_name', 'name', 'is_forward', 'is_midfielder', 'is_goalkeeper', 'is_defense']]
all_gws

,player_index,week_raw,cost_raw,team_name,name,is_forward,is_midfielder,is_goalkeeper,is_defense
46146,1,1,55,west_ham,aaron_cresswell,0,0,0,1
46147,2,1,45,west_ham,adrián_san_miguel_del_castillo,0,0,1,0
46148,745,1,70,west_ham,andriy_yarmolenko,0,1,0,0
46149,3,1,55,west_ham,andy_carroll,1,0,0,0
46150,4,1,45,west_ham,angelo_ogbonna,0,0,0,1
...,...,...,...,...,...,...,...,...,...
67931,823,38,50,fullham,stefan_johansen,0,1,0,0
67932,114,38,45,fullham,timothy_fosu-mensah,0,0,0,1
67933,824,38,42,fullham,tim_ream,0,0,0,1
67934,825,38,46,fullham,tom_cairney,0,1,0,0


In [0]:
players = pd.merge(player_weeks, all_gws, how='inner', left_on=['player_id', 'week'], right_on=['player_index', 'week_raw'])
players = players.drop(columns=['week_raw', 'player_index'])
players['previous_week'] = np.repeat(0, len(players))
players['buy_price'] = np.repeat(0, len(players))
players

,player_id,week,total_points,season,cost_pred,cost_real,long_pred,long_real,short_pred,short_real,cost_raw,team_name,name,is_forward,is_midfielder,is_goalkeeper,is_defense,previous_week,buy_price
0,1,1,0,2018-19,-0.012661,-4.666667,1.544490,0.894737,2.242678,1.666667,55,west_ham,aaron_cresswell,0,0,0,1,0,0
1,1,25,2,2018-19,-0.005661,1.333333,1.853603,0.918919,3.614124,3.000000,50,west_ham,aaron_cresswell,0,0,0,1,0,0
2,1,19,2,2018-19,0.003674,0.000000,1.148359,0.888889,2.846313,2.250000,51,west_ham,aaron_cresswell,0,0,0,1,0,0
3,1,37,0,2018-19,-0.006991,1.333333,0.500508,0.857143,1.528017,0.583333,50,west_ham,aaron_cresswell,0,0,0,1,0,0
4,1,6,0,2018-19,-0.008576,-2.666667,0.564175,0.882353,1.399779,0.833333,53,west_ham,aaron_cresswell,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15722,865,25,2,2018-19,-0.009120,0.000000,0.853458,2.000000,1.801127,2.000000,46,newcastle,fabian_schär,0,0,0,1,0,0
15723,866,25,0,2018-19,0.001557,0.000000,0.632868,0.000000,1.119330,0.000000,56,newcastle,yoshinori_muto,1,0,0,0,0,0
15724,867,37,2,2018-19,0.012554,0.000000,2.531351,2.000000,4.955649,2.000000,59,liverpool,alisson_ramses_becker,0,0,1,0,0,0
15725,868,37,2,2018-19,0.004106,0.000000,1.003423,2.000000,4.630850,2.000000,55,liverpool,fabio_henrique_tavares,0,1,0,0,0,0


## Setting up stats for optimizing on

In [0]:
import itertools

def get_team_memberships(dataframe):

  # A list of all teams
  teams = list(set(dataframe['team_name']))

  # Create a sparse mapping from team id and player id to membership
  teams_list = [
    (
      # Team index
      index,
    
      # Membership map
      dict(zip(
        # Select all players for a team
        list(dataframe[dataframe['team_name'] == teams[index]]['player_id']),
        itertools.repeat(1) # 1 means that it exist
      ))
    ) 
    for index in range(len(teams)) # for every team
  ]
  teams_dict = dict(teams_list)

  return teams_dict

teams_dict = get_team_memberships(players)

## Defining constraints

In [0]:
# Constraint constants
# total_cost = 1000
non_free_transferes = 0

total_forwards = 3
total_midfielder = 5
total_defense = 5
total_goalkeepers = 2
max_in_one_team = 3
number_on_bench = 4

# Tuning params
alpha = 0.5
beta = 0.8
gamma = 0.3

## Define and solve the optimization problem
- x_i equals to 1 iff the player i was in the squad in the previous gameweek and is a starter in the current gameweek, 
- y_i equals to 1 iff the player was in the squad in the previous gameweek and is a substitute in the current one, 
- k_i equals to 1 iff the player i came on a free transfer in the current gameweek and is a starter, 
- l_i equals to 1 iff the player i came on a free transfer in the current gameweek and is a substitute,
- m_i equals to 1 iff the player i came on a paid transfer in the current gameweek and is a starter

Optimize:
\begin{align}
\begin{split}
points = \sum_{i\in I} (
&pts^{long}_{i} (x_i + k_i) + (pts^{long}_{i} − 4) m_i \\ 
&+~\alpha \times (pts^{long}_{i}(y_i + l_i)) \\
&+\beta \times (pts^{short}_{i} (x_i + k_i) + (pts^{short}_{i} − 4) m_i) \\
&+~\gamma \times \Delta cost_i \times (x_i + y_i + k_i + l_i + m_i)
)
\end{split}
\end{align}


If $\alpha = 0$ it means that we do not account for players that are substitutes, $\beta = 0$ means that we don't account for short term fluctuations and $\gamma = 0$ means that we don't account for value changes.

Constraints on: 
- the maximum total number of players in the roster
- the size of the starting squad
- the maximum total number of players representing the same Premier League team, 
- the total number of players having the same position
- the maximum total cost of the whole team imposed by the FPL rules

In [0]:
def solve_for_week(p_df, week, free_transferes = 1, total_budget = 1000):
  player_info = p_df[p_df['week'] == week].drop_duplicates('player_id')

  # Primary variable
  player_identifiers = list(player_info['player_id'])

  # Data fields
  players_points          = dict(zip(player_identifiers, player_info['long_pred']))
  players_points_short    = dict(zip(player_identifiers, player_info['short_pred']))
  player_cost_delta       = dict(zip(player_identifiers, player_info['cost_pred']))
  players_cost            = dict(zip(player_identifiers, player_info['cost_raw']))

  players_is_forward      = dict(zip(player_identifiers, player_info['is_forward']))
  players_is_midfielder   = dict(zip(player_identifiers, player_info['is_midfielder']))
  players_is_goalkeeper   = dict(zip(player_identifiers, player_info['is_goalkeeper']))
  players_is_defense      = dict(zip(player_identifiers, player_info['is_defense']))

  # Is in the previous week of games (limit trading)
  players_prev_week       = dict(zip(player_identifiers, player_info['previous_week']))

  # Variables
  # ------------------------------------------------------------------------------
  prob = LpProblem("select_best_team", LpMaximize)

  # Active players
  x = LpVariable.dicts("x", player_identifiers, cat='Binary')
  k = LpVariable.dicts("k", player_identifiers, cat='Binary')
  m = LpVariable.dicts("m", player_identifiers, cat='Binary')

  # Bench players
  y = LpVariable.dicts("y", player_identifiers, cat='Binary')
  l = LpVariable.dicts("l", player_identifiers, cat='Binary')

  # Main objective
  # ------------------------------------------------------------------------------
  # prob += lpSum([ 
  #   (players_points[i] * (x[i] + k[i]) +
  #   (players_points[i] - 4) * m[i] +
  #    epsilon * players_points[i] *(y[i] + l[i])) 
  #    for i in player_identifiers
  # ])

  # Maybe we should put the value as a constraint instead of affecting the total?
  prob += lpSum([ 
    (
      players_points[i] * (x[i] + k[i]) + (players_points[i] - 4) * m[i] +
      alpha * players_points[i] *(y[i] + l[i]) +
      beta * (players_points_short[i] * (x[i] + k[i]) + (players_points_short[i] - 4) * m[i]) +
      gamma * player_cost_delta[i] * (x[i] + k[i] + m[i] + y[i] + l[i])
      )
    for i in player_identifiers
  ])


  # Constraints
  # ------------------------------------------------------------------------------
  # Max players (and only one of each)
  prob += lpSum([(x[i] + k[i] + m[i] + y[i] + l[i]) for i in player_identifiers]) == 15

  for i in player_identifiers:
    prob += (x[i] + k[i] + m[i] + y[i] + l[i]) <= 1

  # Max cost
  prob += lpSum([players_cost[i] * (x[i] + k[i] + m[i] + y[i] + l[i]) for i in player_identifiers]) <= total_budget

  # Number of transferes we can perform
  prob += lpSum([k[i] + l[i] for i in player_identifiers]) <= free_transferes
  prob += lpSum([m[i] for i in player_identifiers]) <= non_free_transferes
  prob += lpSum([y[i] + l[i] for i in player_identifiers]) == number_on_bench

  # Make sure that x represents what it should
  # x can only be values that were in previous week. k and m can only not be
  prob += lpSum([(1 - players_prev_week[i]) * (x[i] + y[i]) for i in player_identifiers]) == 0
  prob += lpSum([players_prev_week[i] * (k[i] + m[i] + l[i]) for i in player_identifiers]) == 0

  # Accounting for player positions
  prob += lpSum([players_is_forward[i] * (x[i] + k[i] + m[i] + y[i] + l[i]) for i in player_identifiers]) == total_forwards
  prob += lpSum([players_is_midfielder[i] * (x[i] + k[i] + m[i] + y[i] + l[i]) for i in player_identifiers]) == total_midfielder
  prob += lpSum([players_is_defense[i] * (x[i] + k[i] + m[i] + y[i] + l[i]) for i in player_identifiers]) == total_defense
  prob += lpSum([players_is_goalkeeper[i] * (x[i] + k[i] + m[i] + y[i] + l[i]) for i in player_identifiers]) == total_goalkeepers

  # Max number of players representing each team
  # Getting if the player is a member from the sparse mapping from team to players
  for team_index in range(len(teams)):
    prob += lpSum(teams_dict[team_index].get(i, 0) * (x[i] + k[i] + m[i] + y[i] + l[i]) for i in player_identifiers) <= max_in_one_team


  # Solving problem
  # ------------------------------------------------------------------------------

  # Can we have an initial guess?
  success = prob.solve()

  if success > 0:
    print("Successfully for week: " +str(week) + " with free transferes: " + str(free_transferes), " and budget: ", total_budget, "Status:", LpStatus[prob.status])
  else:
    print("Failed for week: " +str(week) + " with free transferes: " + str(free_transferes), "and budget:", total_budget, "Status:", LpStatus[prob.status])

  return prob, success

def create_team_from_solved_problem(prob):
  selected_players = []
  for v in prob.variables():
      if v.varValue > 0:
          selected_players.append((v.name.split('_')[0], v.name.split('_')[1]))

  return selected_players

## Showing new team

In [0]:
def get_sold_players(previous_team, new_team):
  p_team = [int(player_id) for (status, player_id) in previous_team]
  n_team = [int(player_id) for (status, player_id) in new_team]
  return set(p_team) - set(n_team)

def get_new_players(previous_team, new_team):
  p_team = [int(player_id) for (status, player_id) in previous_team]
  n_team = [int(player_id) for (status, player_id) in new_team]
  return set(n_team) - set(p_team)


def solve_for_season():
  teams_for_week = []
  total_performance = 0
  total_budget = 1000
  n_weeks = 38
  free_trades = [1] * (n_weeks + 1) 
  free_trades[1] = 15
  free_trades[10] = 15
  free_trades[29] = 15

  players.loc[:, 'previous_week'] = 0

  old_team = []

  for i in range(1, 39):
    print("")
    solution, success = solve_for_week(players, i, free_trades[i], total_budget)

    if success > 0:
      team = create_team_from_solved_problem(solution)

      new_players = get_new_players(old_team, team)
      sold_players = get_sold_players(old_team, team)

      # Save the buy price
      players.loc[(players['week'] == i) & (players['player_id'].isin(new_players)), 'buy_price'] = players[(players['week'] == i) & (players['player_id'].isin(new_players))]['cost_raw']


      # Buying stats
      print("\nNew players!")
      print('------------------------------------')
      print(players[(players['buy_price'] > 0) & (players['player_id'].isin(new_players))][['buy_price', 'name', 'player_id']].drop_duplicates('player_id'))

      # Selling stats
      print('\nSelling player!')
      print('------------------------------------')
      print('Bought for:')
      print(players[(players['buy_price'] > 0) & (players['player_id'].isin(sold_players))][['buy_price', 'name', 'player_id']].drop_duplicates('player_id'))
      print('Current price:')
      print(players[(players['week'] <= i) & (players['player_id'].isin(sold_players))].sort_values(by='week')[['cost_raw', 'name', 'player_id']].drop_duplicates('player_id', keep='last'))
      print('------------------------------------')

      # Calculating change to budget
      diff_list = players[(players['week'] <= i) & (players['player_id'].isin(sold_players))].sort_values(by='week').drop_duplicates('player_id', keep='last')[['cost_raw']].values - players[(players['buy_price'] > 0) & (players['player_id'].isin(sold_players))].drop_duplicates('player_id')[['buy_price']].values
      diff_list = [val[0] for val in diff_list]
      for price_diff in diff_list:
        if price_diff > 0:
          total_budget += price_diff / 2
        else:
          total_budget -= price_diff
      
      # Remove the price
      players.loc[(players['buy_price'] > 0) & (players['player_id'].isin(sold_players)), 'buy_price'] = 0

      old_team = team
    else:
      team = old_team
      free_trades[i + 1] += free_trades[i]

    player_ids = set([int(player_id) for (_status, player_id) in team])
    players.loc[:, 'previous_week'] = 0
    players.loc[players['player_id'].isin(player_ids), 'previous_week'] = 1

    teams_for_week.append(team)

    team_not_on_bench = set([int(player_id) for (status, player_id) in team if status in set(['x', 'k', 'm'])])

    week_performance = sum(players[(players['week'] == i) & (players['player_id'].isin(team_not_on_bench))]['total_points'])
    total_performance += week_performance
    print("Total Performance for Week: ", week_performance)

  return total_performance, teams_for_week

total_performance, teams_for_week = solve_for_season()


Successfully for week: 1 with free transferes: 15  and budget:  1000 Status: Optimal

New players!
------------------------------------
       buy_price                                     name  player_id
726           45                             josé_holebas         19
2159          99                               sadio_mané         56
2880          60                         andrew_robertson         74
3308          90                              jamie_vardy         85
4127          50                                luke_shaw        105
7100         110                            sergio_agüero        177
7392          55                      patrick_van_aanholt        185
8746          70                         marko_arnautovic        220
9549          55                              ryan_fraser        240
9828          60                          roberto_pereyra        247
10461         64                            marcos_alonso        260
13799         45                   

In [0]:
total_performance


2769

## Some stats regarding transferes

In [0]:
print("Kept, Free, Payed")

n_kept_total = 0
n_free_total = 0
n_payed_total = 0

for week, team in enumerate(teams_for_week):
  print("Week: ", week + 1)
  n_kept = 0
  n_free = 0
  n_payed = 0
  for player in team:
    status, player_id = player

    if status in set(['k', 'l']):
      n_free += 1
    elif status in set(['m']):
      n_payed += 1
    else:
      n_kept += 1

  print(n_kept, n_free, n_payed)

  n_kept_total += n_kept
  n_free_total += n_free
  n_payed_total += n_payed

print("Total: ")
print(n_kept_total, n_free_total, n_payed_total)

Kept, Free, Payed
Week:  1
0 15 0
Week:  2
14 1 0
Week:  3
14 1 0
Week:  4
14 1 0
Week:  5
14 1 0
Week:  6
14 1 0
Week:  7
14 1 0
Week:  8
14 1 0
Week:  9
14 1 0
Week:  10
2 13 0
Week:  11
14 1 0
Week:  12
14 1 0
Week:  13
14 1 0
Week:  14
14 1 0
Week:  15
14 1 0
Week:  16
14 1 0
Week:  17
14 1 0
Week:  18
14 1 0
Week:  19
14 1 0
Week:  20
14 1 0
Week:  21
14 1 0
Week:  22
14 1 0
Week:  23
14 1 0
Week:  24
14 1 0
Week:  25
14 1 0
Week:  26
14 1 0
Week:  27
14 1 0
Week:  28
13 2 0
Week:  29
3 12 0
Week:  30
14 1 0
Week:  31
14 1 0
Week:  32
13 2 0
Week:  33
13 2 0
Week:  34
13 2 0
Week:  35
14 1 0
Week:  36
14 1 0
Week:  37
14 1 0
Week:  38
14 1 0
Total: 
491 79 0


In [0]:
def print_player_stats(player_ids):
  stats = players[(players['week'] == i) & (players['player_id'].isin(set(player_ids)))][['name', 'team_name']].values
  for stat in stats:
    print('   ', end='')
    for val in stat:
      print(val, end='\t\t')
    print('')

players_last_week = []

for week, team in enumerate(teams_for_week):
  print("----------------------------------------------")
  print("Week: ", week + 1)
  x = []
  y = []
  k = []
  l = []
  m = []
  this_week = []

  for player in team:
    status, player_id = player
    this_week.append(player_id)

    if status == 'x':
      x.append(player_id)
    elif status == 'y':
      y.append(player_id)
    elif status == 'k':
      k.append(player_id)
    elif status == 'l':
      l.append(player_id)
    elif status == 'm':
      m.append(player_id)

  print("New transferes in (free):")
  print_player_stats(k)
  if (len(l) > 0):
    print("New transferes to the bench (free)")
    print_player_stats(l)
  if (len(m) > 0):
    print("New transferes in (payed)")
    print_player_stats(m)
  if (len(y) > 0):
    print("Benched players from team:")
    print_player_stats(y)
  removed_players = set(players_last_week) - set(this_week)
  players_last_week = this_week
  if (len(removed_players) > 0):
    print("Removed players:")
    print_player_stats(removed_players)


----------------------------------------------
Week:  1
New transferes in (free):
   josé_holebas		watford		
   sadio_mané		liverpool		
   andrew_robertson		liverpool		
   luke_shaw		manchester_united		
   patrick_van_aanholt		crystal_palace		
   ryan_fraser		bournemouth		
   roberto_pereyra		watford		
   marcos_alonso		chelsea		
   richarlison_de_andrade		everton		
   bernardo_mota_veiga_de_carvalho_e_silva		manchester_city		
   ederson_santana_de_moraes		manchester_city		
New transferes to the bench (free)
   jamie_vardy		leicester		
   sergio_agüero		manchester_city		
   marko_arnautovic		west_ham		
   mathew_ryan		brighton		
----------------------------------------------
Week:  2
New transferes in (free):
   steve_cook		bournemouth		
Benched players from team:
   josé_holebas		watford		
   jamie_vardy		leicester		
   mathew_ryan		brighton		
   bernardo_mota_veiga_de_carvalho_e_silva		manchester_city		
Removed players
   patrick_van_aanholt		crystal_palace		
------------------------